In [98]:
from dashboard.datasource import engine
from dashboard.data import companiesReputation, datasets

In [2]:
import pandas as pd
import plotly.express as px

In [114]:
query = """SELECT PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY position.salary) median,
                                   name,
                                   culture_score,
                                   work_life_balance_score,
                                   career_opportunities_score,
                                   perks_score
                            FROM position
                                     INNER JOIN company on (position.company_id = company.id_company)
                            WHERE total_ratings > 10 AND salary > 0
                            GROUP BY name, culture_score, work_life_balance_score, career_opportunities_score, perks_score, total_ratings
                            order by culture_score DESC
                            LIMIT 5
                            """

queryTotalReviews = """select to_char(review_date,'Mon') as month,
        EXTRACT(month from review_date) AS month_number,
       COUNT(*)
from user_review
group by 1, 2
ORDER BY month_number"""

queryYear = """select to_char(review_date,'YYYY') as year,
       COUNT(*)
from user_review
group by 1
ORDER BY year"""

queryX = """
select to_char(date_position,'YYYY MM') as year,
       to_char(date_position,'YYYY mon') as date,
       COUNT(*)
from position
where to_char(date_position,'YYYY MM') >= '2021 10'
group by 1, 2
ORDER BY 1
    """

QUERYY = """
SELECT remote, count(*) from position GROUP by remote
"""

In [115]:
with engine.connect() as conn:
    df = pd.read_sql_query(QUERYY, conn)

In [4]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [109]:
fig = px.bar(
    df,
    x='name',
    y='avg_reputation',
    range_y=[0, 5],
    title='Average Reputation',
    color='name',
    text_auto=True,
)
fig.update_layout(
    title_text="Average Reputation",
    template="plotly_dark",
    width=800, )
fig.show()

In [15]:
px.data.iris()

,sepal_length,sepal_width,petal_length,petal_width,species,species_id
0,5.1,3.5,1.4,0.2,setosa,1
1,4.9,3.0,1.4,0.2,setosa,1
2,4.7,3.2,1.3,0.2,setosa,1
3,4.6,3.1,1.5,0.2,setosa,1
4,5.0,3.6,1.4,0.2,setosa,1
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica,3
146,6.3,2.5,5.0,1.9,virginica,3
147,6.5,3.0,5.2,2.0,virginica,3
148,6.2,3.4,5.4,2.3,virginica,3


In [104]:
fig = px.line(df, x='date', y="count")
fig.show()

In [127]:
fig = go.Figure(data=[go.Pie(labels=['No Remote','Remote'], values=df['count'])])
fig.update_traces(textinfo='label+percent', textfont_size=20, )
fig.update_layout(
    title_text="Remote Works",
    template="plotly_dark")
fig.show()